In [9]:
import pandas as pd
import numpy as np
import folium
import math
import joblib


In [12]:
# Load model and scaler
model = joblib.load("C:\\Relevant\\College\\Projects\\Cyfuture\\demand_model.joblib")
scaler = joblib.load("C:\\Relevant\\College\\Projects\\Cyfuture\\scaler.joblib")

# Load required data
regions_df = pd.read_csv("C:\\Relevant\\College\\Projects\\Cyfuture\\delhi_region_landmark_distances_corrected.csv")

features = ['day_of_week', 'month', 'holiday', 'temperature', 'humidity',
            'wind_speed', 'rainfall', 'dist_city', 'dist_airport', 'dist_railway']
weather_features = ['temperature', 'humidity', 'wind_speed', 'rainfall']

# Prompt user for dates input (comma-separated)
input_dates = input("Enter one or more dates (YYYY-MM-DD), separated by commas: ")

# Process input into a list of date strings, stripped of whitespace
dates_to_predict = [date.strip() for date in input_dates.split(',') if date.strip()]

print(f"Dates to predict: {dates_to_predict}")


Enter one or more dates (YYYY-MM-DD), separated by commas:  2025-03-28


Dates to predict: ['2025-03-28']


In [13]:
def get_calendar_features(date_str):
    import datetime
    dt = datetime.datetime.strptime(date_str, "%Y-%m-%d")
    return {
        'day_of_week': dt.weekday(),
        'month': dt.month,
        'holiday': 0   # Or logic if you have holidays data
    }

def generate_random_weather():
    return {
        'temperature': np.random.uniform(20, 40),
        'humidity': np.random.uniform(30, 70),
        'wind_speed': np.random.uniform(5, 15),
        'rainfall': np.random.uniform(0, 10)
    }

def destination_point(lat, lon, distance_m, bearing_deg):
    R = 6371000
    bearing = math.radians(bearing_deg)
    lat1, lon1 = math.radians(lat), math.radians(lon)
    lat2 = math.asin(math.sin(lat1)*math.cos(distance_m/R) + math.cos(lat1)*math.sin(distance_m/R)*math.cos(bearing))
    lon2 = lon1 + math.atan2(math.sin(bearing)*math.sin(distance_m/R)*math.cos(lat1), math.cos(distance_m/R)-math.sin(lat1)*math.sin(lat2))
    return math.degrees(lat2), math.degrees(lon2)

def create_map_for_date(date_str, predictions_df):
    delhi_center = (28.6139, 77.2090)
    R = 30000
    R1 = R / math.sqrt(31)
    ring_radii = [R1 * math.sqrt(2**k - 1) for k in range(1, 6)]
    sectors_per_ring = [4 * 2**(k - 1) for k in range(1, 6)]
    m = folium.Map(location=delhi_center, zoom_start=11)
    region_num = 1

    for i, r_outer in enumerate(ring_radii):
        r_inner = 0 if i == 0 else ring_radii[i - 1]
        sectors = sectors_per_ring[i]
        sector_angle = 360 / sectors

        for sector in range(sectors):
            start_angle = sector * sector_angle
            end_angle = (sector + 1) * sector_angle
            points = []

            for angle in np.linspace(start_angle, end_angle, num=10):
                lat, lon = destination_point(delhi_center[0], delhi_center[1], r_inner, angle)
                points.append((lat, lon))
            for angle in np.linspace(end_angle, start_angle, num=10):
                lat, lon = destination_point(delhi_center[0], delhi_center[1], r_outer, angle)
                points.append((lat, lon))

            demand_val = predictions_df.loc[predictions_df['region'] == region_num, 'predicted_demand'].values
            demand_val = demand_val[0] if len(demand_val) > 0 else 0
            color = 'red' if demand_val == 1 else 'blue'

            folium.Polygon(
                locations=points,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.4,
                weight=1,
                tooltip=f'Region {region_num}: Demand={"High" if demand_val == 1 else "Low"} - Date: {date_str}'
            ).add_to(m)

            region_num += 1

    return m


In [14]:
for date_str in dates_to_predict:
    records = []
    calendar_features = get_calendar_features(date_str)
    weather_vals = generate_random_weather()

    for _, region in regions_df.iterrows():
        record = {
            'region': region['region'],
            'dist_city': region['dist_city'],
            'dist_airport': region['dist_airport'],
            'dist_railway': region['dist_railway'],
            **calendar_features,
            **weather_vals
        }
        records.append(record)

    df_pred = pd.DataFrame(records)

    # Scale weather features
    df_pred[weather_features] = scaler.transform(df_pred[weather_features])

    X_pred = df_pred[features]
    preds = model.predict(X_pred)
    df_pred['predicted_demand'] = preds

    print(f"\n--- Demand Map for Date: {date_str} ---")
    m = create_map_for_date(date_str, df_pred)
    display(m)



--- Demand Map for Date: 2025-03-28 ---
